In [262]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import kaleido
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import  silhouette_score


In [263]:
df = pd.read_csv('uber_data_sample.csv')


## Base et outils d'analyse
Nous avons identifié que le jeudi est la journée la plus dense et que 17h est l’heure la plus active.
Nous allons utiliser ces deux critères pour nos modèles de machine learning non supervisé.

Nous commencerons avec KMEANS, un modèle basé sur le calcul de centroïdes et utilisant la distance euclidienne. KMEANS cherche à regrouper les points autour du centroïde en minimisant la moyenne des distances, afin de créer des clusters compacts.

Ensuite, nous enchaînerons avec DBSCAN, un modèle basé sur le concept de densité. Selon une mesure de distance (euclidienne ou Manhattan), DBSCAN considère deux paramètres : eps (rayon du voisinage) et min_samples (nombre minimal de points pour former un cluster). Si le nombre de points dans le voisinage est supérieur ou égal à min_samples, le point est intégré au cluster ; sinon, il est considéré comme bruit ou déclenche un nouveau cluster.

In [264]:
# Nos paramètres pour le modèle
day = 'Thursday'
hours = 17

### KMEANS Jour

In [265]:
# On filtre le DataFrame pour ne garder que le jour sélectionné.
# Pas de preprocessing nécessaire car le clustering KMEANS utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_day=df['dayname'] == day
df_day = df[mask_day]
df_day1 = df_day[['Lat', 'Lon']]

# Calcul du WCSS : méthode du coude
wcss =  []
for i in range (2,30):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_day1)
    wcss.append(kmeans.inertia_)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))


WCSS for K=2 --> 19.532886177082897
WCSS for K=3 --> 15.253735677166349
WCSS for K=4 --> 11.872315597749637
WCSS for K=5 --> 9.109068150378196
WCSS for K=6 --> 7.349316343632216
WCSS for K=7 --> 6.137286530062129
WCSS for K=8 --> 5.4445540448531
WCSS for K=9 --> 4.726459351066321
WCSS for K=10 --> 4.236836632693791
WCSS for K=11 --> 3.902605871711579
WCSS for K=12 --> 3.3426792850729106
WCSS for K=13 --> 2.9592249306212666
WCSS for K=14 --> 2.8341040703497695
WCSS for K=15 --> 2.690783403097668
WCSS for K=16 --> 2.5325982025523066
WCSS for K=17 --> 2.3529149560613525
WCSS for K=18 --> 2.2464982653351226
WCSS for K=19 --> 2.135985177234784
WCSS for K=20 --> 2.0556754816035006
WCSS for K=21 --> 1.8625040720356838
WCSS for K=22 --> 1.7063961615127952
WCSS for K=23 --> 1.598044779192208
WCSS for K=24 --> 1.5181225679428028
WCSS for K=25 --> 1.4542205834696078
WCSS for K=26 --> 1.3947700298641605
WCSS for K=27 --> 1.3599774242995872
WCSS for K=28 --> 1.3117025094031025
WCSS for K=29 --> 1.2

In [266]:
fig = px.line(x = range(2,30), y = wcss)
fig.show()

Forte diminution jusqu’à K10.
Au-delà de K10, la pente de la courbe s’adoucit progressivement, indiquant un retour décroissant de l’ajout de clusters supplémentaires.

Notre coude semble se situer entre K10 et K12.

In [267]:
# Calcul du score de silhouette
sil = []
k = []

for i in range (2,30):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_day1)
    sil.append(silhouette_score(df_day1, kmeans.predict(df_day1)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7204098726787465
Silhouette score for K=3 is 0.37927419999374345
Silhouette score for K=4 is 0.39848972613178774
Silhouette score for K=5 is 0.4199982455606189
Silhouette score for K=6 is 0.4521601015374075
Silhouette score for K=7 is 0.4557896644192709
Silhouette score for K=8 is 0.42161180132570586
Silhouette score for K=9 is 0.42306796579510497
Silhouette score for K=10 is 0.44135050919689633
Silhouette score for K=11 is 0.38890346501595446
Silhouette score for K=12 is 0.3930373729349778
Silhouette score for K=13 is 0.3984104788977687
Silhouette score for K=14 is 0.3963269046705927
Silhouette score for K=15 is 0.395235000039253
Silhouette score for K=16 is 0.3958738476094415
Silhouette score for K=17 is 0.4037429413209185
Silhouette score for K=18 is 0.3829163421793455
Silhouette score for K=19 is 0.3821304883534233
Silhouette score for K=20 is 0.38619304614858146
Silhouette score for K=21 is 0.4029396158302642
Silhouette score for K=22 is 0.40338687697

In [268]:
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

fig = px.bar(data_frame=cluster_scores,
             x=k,
             y=cluster_scores.iloc[:, -1]
            )

fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="Clusters",
    title="Silhouette Score par cluster"
)

fig.show()

Le score est très élevé pour K2, mais les clusters sont trop globaux pour une utilisation opérationnelle.
Entre K6 et K10, le score remonte légèrement, suggérant un bon compromis entre compacité et séparation.
Au-delà de K10, le score se stabilise autour de 0.40–0.41, ce qui indique que l’ajout de clusters supplémentaires n’améliore pas vraiment la qualité.

K10 semble optimal pour ce jeu de données :

- Permet de créer des clusters suffisamment détaillés pour distinguer des zones géographiques pertinentes.

- Maintient une bonne compacité et séparation.

Ce choix offre un bon compromis entre granularité opérationnelle et qualité statistique des clusters 

In [269]:
kmeans = KMeans(n_clusters=10, random_state=42)
kmeans.fit(df_day1)
df_day1 = df_day1.copy()
df_day1.loc[:, 'Kmeans'] = kmeans.predict(df_day1)

fig = px.scatter_map(
        df_day1,
        lat="Lat",
        lon="Lon",
        color="Kmeans",
        map_style="carto-positron",
        zoom=9,
        title= f"HOT POINT {day}"
)

fig.show()

### Analyse de densité et visualisation des clusters

Nous affichons les centres des clusters avec des cercles dont la taille est proportionnelle à la densité des trajets. Cela permet d’identifier rapidement les zones les plus actives et stratégiques pour Uber à un jour et une heure donnés.

In [270]:
# Calculer le centre de chaque cluster
cluster_summary = df_day1.groupby('Kmeans').agg(
    Lat_center=('Lat', 'mean'),
    Lon_center=('Lon', 'mean'),
    density=('Kmeans', 'count')
).reset_index()

# Normaliser la densité pour que la taille des cercles soit proportionnelle
cluster_summary['density_norm'] = cluster_summary['density'] / cluster_summary['density'].max() * 100  # facteur d'échelle


fig = px.scatter_map(
    cluster_summary,
    lat="Lat_center",
    lon="Lon_center",
    size="density_norm",      
    color="Kmeans",           
    hover_name="Kmeans",
    hover_data={"density": True, "Lat_center": False, "Lon_center": False},
    zoom=9,
    map_style="carto-positron",
    title=f"Densité des clusters sur {day}"
)

fig.show()

In [271]:
ratio_densite_max = (cluster_summary['density'].max() * 100 / cluster_summary['density'].sum()).round(2)

print(f'le cluster {cluster_summary.loc[cluster_summary["density"].idxmax(), "Kmeans"]} représente {ratio_densite_max}% de la densité totale des clusters')

le cluster 6 représente 37.0% de la densité totale des clusters


### KMEANS heure

In [272]:
# On filtre le DataFrame pour ne garder que l'heure sélectionnée.
# Pas de preprocessing nécessaire car le clustering KMEANS utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_hour=df['hour'] == hours
df_hour = df[mask_hour]
df_hour1 = df_hour[['Lat', 'Lon']]

# Calcul du WCSS : méthode du coude
wcss =  []
for i in range (2,70):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_hour1)
    wcss.append(kmeans.inertia_)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

WCSS for K=2 --> 8.753779693353321
WCSS for K=3 --> 6.778536923554691
WCSS for K=4 --> 5.653443109200355
WCSS for K=5 --> 4.198260116993216
WCSS for K=6 --> 3.5239770637319907
WCSS for K=7 --> 2.851821742969376
WCSS for K=8 --> 2.4971001379828235
WCSS for K=9 --> 2.3015482416258854
WCSS for K=10 --> 2.0477303971916174
WCSS for K=11 --> 1.9239360416319111
WCSS for K=12 --> 1.701616092186207
WCSS for K=13 --> 1.4306412665587063
WCSS for K=14 --> 1.2980185527425998
WCSS for K=15 --> 1.1826713812545064
WCSS for K=16 --> 1.0331159034403212
WCSS for K=17 --> 0.9807222582782429
WCSS for K=18 --> 0.923589292508826
WCSS for K=19 --> 0.8627568757273844
WCSS for K=20 --> 0.813189162172375
WCSS for K=21 --> 0.7439689719535716
WCSS for K=22 --> 0.7112820040019675
WCSS for K=23 --> 0.6735887098352978
WCSS for K=24 --> 0.6341731441248899
WCSS for K=25 --> 0.5952216996082269
WCSS for K=26 --> 0.5571010308015283
WCSS for K=27 --> 0.5302377517949087
WCSS for K=28 --> 0.5198861256340047
WCSS for K=29 -->

In [273]:
fig = px.line(x = range(2,70), y = wcss)
fig.show()

La chute est forte entre K2 et K10. La pente s'adoucit à partir de K16 et diminue de manière plus légère et progressive.
Le coude semble donc se situer entre K10 et K15

In [274]:
# Calcul du score de silhouette
sil = []
k = []

for i in range (2,70):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_hour1)
    sil.append(silhouette_score(df_hour1, kmeans.predict(df_hour1)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7211764285468715
Silhouette score for K=3 is 0.41250398196932286
Silhouette score for K=4 is 0.43885264366902194
Silhouette score for K=5 is 0.4478952238121377
Silhouette score for K=6 is 0.4548703289235388
Silhouette score for K=7 is 0.4336293670736129
Silhouette score for K=8 is 0.43176508955902404
Silhouette score for K=9 is 0.44364320095325577
Silhouette score for K=10 is 0.44415922388057844
Silhouette score for K=11 is 0.4446295452618548
Silhouette score for K=12 is 0.4394612735395502
Silhouette score for K=13 is 0.440330689078259
Silhouette score for K=14 is 0.4428981600459362
Silhouette score for K=15 is 0.4438781128567472
Silhouette score for K=16 is 0.4047846260662981
Silhouette score for K=17 is 0.40312994438874966
Silhouette score for K=18 is 0.4072463324859124
Silhouette score for K=19 is 0.4088638156668238
Silhouette score for K=20 is 0.41627439742349037
Silhouette score for K=21 is 0.37549842485061447
Silhouette score for K=22 is 0.3790275376

In [275]:
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)


fig = px.bar(data_frame=cluster_scores,
             x=k,
             y=cluster_scores.iloc[:, -1]
            )


fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="Clusters",
    title="Silhouette Score par cluster"
)

fig.show()

Nous observons un pic élevé du score de silhouette à K2, ce qui indique des clusters bien séparés mais très globaux.
Entre K10 et K15, les scores de silhouette se stabilisent autour de 0.44, ce qui reste un niveau de qualité satisfaisant tout en permettant une segmentation plus détaillée.
Au-delà de K15, le score se dégrade progressivement, ce qui suggère que l’algorithme crée des clusters supplémentaires de manière moins naturelle et moins séparée.

K15 apparaît donc comme un bon compromis :
- assez de finesse pour distinguer des zones géographiques pertinentes,
- tout en conservant une bonne compacité et séparation des clusters.

Dans un contexte opérationnel pour Uber, K15 offrira une segmentation fine et exploitable pour mieux équilibrer l’offre et la demande sur le territoire.


In [276]:
kmeans = KMeans(n_clusters=15, random_state=42)

kmeans.fit(df_hour1)
df_hour1 = df_hour1.copy()
df_hour1.loc[:, 'Kmeans'] = kmeans.predict(df_hour1)

fig = px.scatter_map(
        df_hour1,
        lat="Lat",
        lon="Lon",
        color="Kmeans",
        map_style="carto-positron",
        zoom=9,
        title= f"HOT POINT {hours} H"
)

fig.show()

### Analyse de densité et visualisation des clusters

Nous affichons les centres des clusters avec des cercles dont la taille est proportionnelle à la densité des trajets. Cela permet d’identifier rapidement les zones les plus actives et stratégiques pour Uber à un jour et une heure donnés.

In [277]:
# Calculer le centre de chaque cluster
cluster_summary = df_hour1.groupby('Kmeans').agg(
    Lat_center=('Lat', 'mean'),
    Lon_center=('Lon', 'mean'),
    density=('Kmeans', 'count')
).reset_index()

# Normaliser la densité pour que la taille des cercles soit proportionnelle
cluster_summary['density_norm'] = cluster_summary['density'] / cluster_summary['density'].max() * 100  # facteur d'échelle


fig = px.scatter_map(
    cluster_summary,
    lat="Lat_center",
    lon="Lon_center",
    size="density_norm",      
    color="Kmeans",           
    hover_name="Kmeans",
    hover_data={"density": True, "Lat_center": False, "Lon_center": False},
    zoom=9,
    map_style="carto-positron",
    title=f"Densité des clusters à {hours}"
)

fig.show()

In [278]:
ratio_densite_max = (cluster_summary['density'].max() * 100 / cluster_summary['density'].sum()).round(2)

print(f'le cluster {cluster_summary.loc[cluster_summary["density"].idxmax(), "Kmeans"]} représente {ratio_densite_max}% de la densité totale des clusters')

le cluster 1 représente 27.15% de la densité totale des clusters


### KMEANS Jour et Heure

In [279]:
# On filtre le DataFrame pour ne garder que l'heure et le jour sélectionné.
# Pas de preprocessing nécessaire car le clustering KMEANS utilisera uniquement les colonnes 'Lat' et 'Lon'.
mask_dh = (df['dayname'] == day) & (df['hour'] == hours)
df_dh = df[mask_dh]
df_dh1 = df_dh[['Lat', 'Lon']]

# Calcul du WCSS : méthode du coude
wcss =  []
for i in range (2,70):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_dh1)
    wcss.append(kmeans.inertia_)
    print("WCSS for K={} --> {}".format(i, wcss[-1]))

WCSS for K=2 --> 1.0407831903180715
WCSS for K=3 --> 0.7462846001006371
WCSS for K=4 --> 0.6427888186501833
WCSS for K=5 --> 0.4886803239607018
WCSS for K=6 --> 0.4091482016738947
WCSS for K=7 --> 0.32333958497089105
WCSS for K=8 --> 0.21768926262678642
WCSS for K=9 --> 0.19950278948984973
WCSS for K=10 --> 0.16326761214392163
WCSS for K=11 --> 0.15461103646828908
WCSS for K=12 --> 0.13860326731331435
WCSS for K=13 --> 0.12523625832903282
WCSS for K=14 --> 0.11415863332903209
WCSS for K=15 --> 0.09312421571849057
WCSS for K=16 --> 0.08540202238520649
WCSS for K=17 --> 0.07763206268513992
WCSS for K=18 --> 0.07500765680288293
WCSS for K=19 --> 0.07220923196656984
WCSS for K=20 --> 0.06316770318181963
WCSS for K=21 --> 0.05462159087033483
WCSS for K=22 --> 0.050785944935124325
WCSS for K=23 --> 0.046657445126533914
WCSS for K=24 --> 0.04548386845986745
WCSS for K=25 --> 0.04323356016658418
WCSS for K=26 --> 0.04085321834118761
WCSS for K=27 --> 0.03752173758133101
WCSS for K=28 --> 0.035

In [280]:
fig = px.line(x = range(2,70), y = wcss)
fig.show()

In [281]:
# Calcul du score de silhouette
sil = []
k = []

for i in range (2,70):
    kmeans = KMeans(n_clusters= i, random_state = 0, n_init = 'auto')
    kmeans.fit(df_dh1)
    sil.append(silhouette_score(df_dh1, kmeans.predict(df_dh1)))
    k.append(i)
    print("Silhouette score for K={} is {}".format(i, sil[-1]))

Silhouette score for K=2 is 0.7297512900420027
Silhouette score for K=3 is 0.4505591448893968
Silhouette score for K=4 is 0.36832857198891783
Silhouette score for K=5 is 0.41635912031642974
Silhouette score for K=6 is 0.3769719739144572
Silhouette score for K=7 is 0.3879637841988013
Silhouette score for K=8 is 0.43902256060339767
Silhouette score for K=9 is 0.4411662631441849
Silhouette score for K=10 is 0.42469063364027687
Silhouette score for K=11 is 0.37070484864258746
Silhouette score for K=12 is 0.3592476451237156
Silhouette score for K=13 is 0.387028740687895
Silhouette score for K=14 is 0.3860814944629455
Silhouette score for K=15 is 0.4201656727776441
Silhouette score for K=16 is 0.403150668195799
Silhouette score for K=17 is 0.3955112201461103
Silhouette score for K=18 is 0.369626702667792
Silhouette score for K=19 is 0.3648128100115493
Silhouette score for K=20 is 0.376465749925924
Silhouette score for K=21 is 0.42206031872190997
Silhouette score for K=22 is 0.424412641677316

In [282]:
cluster_scores=pd.DataFrame(sil)
k_frame = pd.Series(k)

fig = px.bar(data_frame=cluster_scores,
             x=k,
             y=cluster_scores.iloc[:, -1]
            )

fig.update_layout(
    yaxis_title="Silhouette Score",
    xaxis_title="Clusters",
    title="Silhouette Score par cluster"
)

fig.show()

D’après les scores obtenus avec la méthode du coude (Elbow) et le silhouette score, le meilleur compromis semble être 9 clusters (score 0,44). Cependant, on constate que 23 clusters donnent un score proche (0,42). En choisissant 23 clusters, nous obtenons davantage de clusters, ce qui permet une meilleure granularité, notamment dans les zones très denses.

In [283]:
kmeans = KMeans(n_clusters=23, random_state=42)
kmeans.fit(df_dh1)
df_dh1 = df_dh1.copy()
df_dh1.loc[:, 'Kmeans'] = kmeans.predict(df_dh1)

fig = px.scatter_map(
        df_dh1,
        lat="Lat",
        lon="Lon",
        color="Kmeans",
        map_style="carto-positron",
        zoom=9,
        title= f"HOT POINT {day} {hours}H"
)

fig.show()

Essayons de donner plus de clarté à notre map en ajoutant un filtre sur le nombre de point minimum par cluster. Cela devrait éliminer le bruit/outliers de certains clusters

In [284]:
# nombre de point par cluster
nb_cluster = df_dh1['Kmeans'].value_counts()
print(nb_cluster)


Kmeans
0     118
10     78
19     58
17     52
8      41
2      36
16     33
18     26
7      21
15     18
1      11
13      9
3       8
4       7
21      4
12      4
20      3
6       3
14      3
22      3
11      2
5       1
9       1
Name: count, dtype: int64


In [285]:
# Nous prendrons un minimum de 50 points par cluster
min_point = 50
df_dh_cluster = df_dh1[df_dh1['Kmeans'].isin(nb_cluster[nb_cluster >= min_point].index)]

fig = px.scatter_map(
        df_dh_cluster,
        lat="Lat",
        lon="Lon",
        color="Kmeans",
        map_style="carto-positron",
        zoom=10,
        title= f"HOT POINT {day} {hours}H avec un minimum de 50 points"
)

fig.show()

Nous avons maintenant un affichage plus clair des zones chaudes. Les différents clusters se distinguent mieux, notamment dans des quartiers emblématiques de New York comme Midtown et le Financial District.
Observons à présent comment réagit le modèle pour les autres jours et heures de la semaine. Dans le modèle suivant, nous utiliserons les mêmes paramètres avec n_clusters = 23.

In [286]:
jours_ordre = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
all_hours = list(range(24))

# Fonction pour créer la figure KMeans
def create_figure(day, hour, n_clusters=23):
    # Filtrage sur jour et heure
    df_plot = df[(df["dayname"] == day) & (df["hour"] == hour)].copy()

    # KMeans sur les coordonnées Lat/Lon
    coords = df_plot[["Lat", "Lon"]].values
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    df_plot.loc[:, "Kmeans"] = kmeans.fit_predict(coords)

    # Création de la carte
    fig = px.scatter_map(
        df_plot,
        lat="Lat",
        lon="Lon",
        color="Kmeans",
        map_style="carto-positron",
        zoom=10,
        width=1000,
        height=700,
        title=f"KMeans Clusters - {day} {hour}h"
    )
    return fig

# Widgets Dropdown pour jour + heure
dropdown_day = widgets.Dropdown(
    options=jours_ordre,
    value="Monday",
    description='Jour:',
)

dropdown_hour = widgets.Dropdown(
    options=all_hours,
    value=12,
    description='Heure:',
)

output = widgets.Output()

# Fonction pour mettre à jour la figure
def update_plot(_):
    with output:
        clear_output(wait=True)
        fig = create_figure(dropdown_day.value, dropdown_hour.value)
        display(fig)

# Observer les changements sur les 2 menus
dropdown_day.observe(update_plot, names="value")
dropdown_hour.observe(update_plot, names="value")

# Affichage initial
with output:
    fig = create_figure(dropdown_day.value, dropdown_hour.value)
    display(fig)

# Affichage des menus + output
display(dropdown_day, dropdown_hour, output)


Dropdown(description='Jour:', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Su…

Dropdown(description='Heure:', index=12, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17…

Output()

Rendons cela plus lisible en utilisant un cercle de densité basé sur le centroïde. Cela permet de visualiser plus rapidement l’importance de chaque cluster par rapport aux autres.

Afin de ne représenter que les clusters les plus significatifs, nous appliquons un filtrage statistique : seuls les clusters dont la densité est supérieure ou égale à la moyenne augmentée d’un écart-type sont conservés.

In [288]:
def create_figure2(days,hr, k_std=1):
    df_plot = df[(df["dayname"] == days) & (df["hour"] == hr)].copy()
    df_plot = df_plot.copy()
    kmeans = KMeans(n_clusters=23, random_state=42)  #
    coords = df_plot[["Lat", "Lon"]]  
    df_plot.loc[:,"Kmeans"] = kmeans.fit_predict(coords)
    
    # Calculer le centre de chaque cluster
    cluster_summary = df_plot.groupby(['Kmeans']).agg(
        Lat_center=('Lat', 'mean'),
        Lon_center=('Lon', 'mean'),
        density=('Kmeans', 'count')
    ).reset_index()

    # Filtrer les clusters par rapport à la moyenne + k*écart-type de densité
    mean_density = cluster_summary['density'].mean()
    std_density = cluster_summary['density'].std()
    threshold = mean_density + k_std * std_density
    cluster_summary = cluster_summary[cluster_summary['density'] >= threshold]

    # Normaliser la densité pour que la taille des cercles soit proportionnelle
    cluster_summary['density_norm'] = cluster_summary['density'] / cluster_summary['density'].max() * 100  # facteur d'échelle

    fig = px.scatter_map(
        cluster_summary,
        lat="Lat_center",
        lon="Lon_center",
        size="density_norm",      
        color="Kmeans",
        hover_name="Kmeans",
        hover_data={"density": True, "Lat_center": False, "Lon_center": False},
        zoom=10,
        map_style="carto-positron",
        width=1000,
        height=700,
        title=f"Densité des clusters {days} {hr}h"
    )
    return fig

# Widgets Dropdown pour jour + heure
dropdown_day = widgets.Dropdown(
    options=jours_ordre,
    value="Monday",
    description='Jour:',
)

dropdown_hour = widgets.Dropdown(
    options=all_hours,
    value=12,
    description='Heure:',
)

output = widgets.Output()

# Fonction pour mettre à jour la figure
def update_plot(_):
    with output:
        clear_output(wait=True)
        fig = create_figure2(dropdown_day.value, dropdown_hour.value)
        display(fig)

# Observer les changements sur les 2 menus
dropdown_day.observe(update_plot, names="value")
dropdown_hour.observe(update_plot, names="value")

# Affichage initial
with output:
    fig = create_figure2(dropdown_day.value, dropdown_hour.value)
    display(fig)

# Affichage des menus + output
display(dropdown_day, dropdown_hour, output)

Dropdown(description='Jour:', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Su…

Dropdown(description='Heure:', index=12, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17…

Output()

Notre dernier modèle, intégrant un filtrage statistique, révèle les clusters les plus significatifs pour chaque jour et chaque heure.
Le cercle de densité offre une lecture immédiate du secteur le plus actif à un moment donné de la journée, ce qui facilite l’identification des zones à forte concentration d’activité.
Au-delà de cette visualisation, le modèle permet de comparer la dynamique spatiale entre les jours de semaine et les week-ends. Le filtrage statistique renforce la robustesse des résultats en écartant les clusters peu significatifs, ce qui améliore l’interprétation et soutient la prise de décision.

### Pour conclure
KMEANS offre plusieurs avantages dans notre cas :

- Il est simple à mettre en place et très rapide, même sur un gros volume de trajets Uber.

- Il permet d’identifier des zones géographiques précises (centroïdes), utiles pour repérer des “hotspots”.

- Il reste intuitif et facile à interpréter.

Cependant, il présente aussi certaines limites :

- Il suppose que les clusters soient “ronds” et séparés de manière euclidienne.

- Or, dans notre contexte, certaines zones peuvent être allongées (axes routiers, couloirs urbains), ce que KMeans gère mal.

- Les points isolés (trajets éloignés) peuvent tirer les centroïdes et introduire du bruit dans la lecture des cartes.

Pour ces raisons, il est pertinent de compléter l’analyse avec un autre modèle : DBSCAN, qui est moins sensible au bruit et mieux adapté aux zones de densité variable.